# SimResults To Race Reports and Championship Tables

In [10]:
import pandas as pd
import numpy as np
import re, os

### Race Report Object

In [11]:
class RaceReport():
    
    # Race report table format strings
    qualy_row_0 = """(% border="1" style="width:554px" %)"""
    qualy_row_1 = """|=(% scope="row" style="border-color: rgb(0, 0, 0); background-color: rgb(234, 236, 240); width: 41px; text-align: center;" %)Pos|=(% style="border-color: rgb(0, 0, 0); background-color: rgb(234, 236, 240); width: 44px; text-align: center;" %)No|=(% style="border-color: rgb(0, 0, 0); background-color: rgb(234, 236, 240); width: 155px; text-align: center;" %)Driver|=(% style="border-color: rgb(0, 0, 0); background-color: rgb(234, 236, 240); width: 233px; text-align: center;" %)Team|=(% style="border-color: rgb(0, 0, 0); background-color: rgb(234, 236, 240); width: 77px; text-align: center;" %)Time"""
    qualy_row_2 = """|=(% style="border-color: rgb(0, 0, 0); background-color: rgb(234, 236, 240); text-align: center; width: 41px;" %){}|(% style="border-color:#000000; text-align:center; width:44px" %){}|(% style="border-color:#000000; width:155px" %)[[image:{}||height="14" width="23"]] {}|(% style="text-align:center; border-color:#000000; width:233px" %){}|(% style="text-align:center; border-color:#000000; width:77px" %){}"""
    qualy_row = """|=(% style="background-color: rgb(234, 236, 240); text-align: center; width: 41px;" %){}|(% style="text-align:center; width:44px" %){}|(% style="width:155px" %)[[image:{}||height="14" width="23"]] {}|(% style="text-align:center; width:233px" %){}|(% style="text-align:center; width:77px" %){}"""
    qualy_row_last = """|=(% colspan="5" style="border-color: rgb(0, 0, 0); background-color: rgb(234, 236, 240); text-align: center; width: 552px;" %)[[Source>>{}]]"""
    race_row_0 = """(% border="1" style="width:747px" %)"""
    race_row_1 = """|=(% scope="row" style="border-color: rgb(0, 0, 0); background-color: rgb(234, 236, 240); width: 41px; text-align: center;" %)Pos|=(% style="border-color: rgb(0, 0, 0); background-color: rgb(234, 236, 240); width: 44px; text-align: center;" %)No|=(% style="border-color: rgb(0, 0, 0); background-color: rgb(234, 236, 240); width: 155px; text-align: center;" %)Driver|=(% style="border-color: rgb(0, 0, 0); background-color: rgb(234, 236, 240); width: 229px; text-align: center;" %)Team|=(% style="border-color: rgb(0, 0, 0); background-color: rgb(234, 236, 240); width: 44px; text-align: center;" %)Laps|=(% style="border-color: rgb(0, 0, 0); background-color: rgb(234, 236, 240); width: 111px; text-align: center;" %)Time/Retired|=(% style="border-color: rgb(0, 0, 0); background-color: rgb(234, 236, 240); width: 53px; text-align: center;" %)Grid|=(% style="border-color: rgb(0, 0, 0); background-color: rgb(234, 236, 240); width: 62px; text-align: center;" %)Points"""
    race_row_2 = """|=(% style="border-color: rgb(0, 0, 0); background-color: rgb(234, 236, 240); text-align: center; width: 41px;" %){}|(% style="border-color:#000000; text-align:center; width:44px" %){}|(% style="border-color:#000000; width:155px" %)[[image:{}||height="14" width="23"]] {}|(% style="text-align:center; border-color:#000000; width:233px" %){}|(% style="text-align:center; border-color:#000000; width:44px" %){}|(% style="text-align:center; border-color:#000000; width:111px" %){}|(% style="text-align:center; border-color:#000000; width:53px" %){}|(% style="text-align:center; border-color:#000000; width:62px" %){}"""
    race_row = """|=(% style="background-color: rgb(234, 236, 240); text-align: center; width: 41px;" %){}|(% style="text-align:center; width:44px" %){}|(% style="width:155px" %)[[image:{}||height="14" width="23"]] {}|(% style="text-align:center; width:233px" %){}|(% style="text-align:center; width:44px" %){}|(% style="text-align:center; width:111px" %){}|(% style="text-align:center; width:53px" %){}|(% style="text-align:center; width:62px" %){}"""
    race_row_last = """|=(% colspan="8" style="border-color: rgb(0, 0, 0); background-color: rgb(234, 236, 240); text-align: center; width: 745px;" %)[[Source>>{}]]"""
    race_row_fastestlap = """Fastest lap:  [[image:{}||height="14" width="23"]] {} - {}"""
    
    
    # Read series drivers info
    def read_drivers_table(series_directory):
        drivers_table_file = "{}/drivers_table.csv".format(series_directory)
        return pd.read_csv(drivers_table_file, dtype=object)
    
    
    # Read series points scoring info
    def read_points_table(series_directory):
        points_table_file = "{}/points_table.csv".format(series_directory)
        return pd.read_csv(points_table_file, dtype=object)
    
    
    # Constructor, optionally pass in already-parsed drivers and points table info, debug to print intermediate dataframes
    def __init__(self, table_names, series_directory, race_directory, output_file_name="wiki_tables.txt", drivers_table=None, points_table=None, debug=False):
        
        self.table_names = table_names
        self.debug = debug
        
        self.drivers_table = drivers_table if drivers_table is not None else RaceReport.read_drivers_table(series_directory)
        self.points_table = points_table if points_table is not None else RaceReport.read_points_table(series_directory)

        self.output_file = "{}/{}/{}".format(series_directory, race_directory, output_file_name)
        
        simresults_file_name = [file for file in os.listdir("{}/{}".format(series_directory, race_directory)) if file.endswith(".csv")][0]
        simresults_code = os.path.splitext(simresults_file_name)[0]
        self.results_file = "{}/{}/{}.csv".format(series_directory, race_directory, simresults_code)
        self.simresults_url = "https://simresults.net/{}".format(simresults_code)
        
        self.tables = self.__read_results_tables()
        self.__clean_results_tables()
    
    
    # Return race result pandas dataframes in dict keyed by table_names
    def __read_results_tables(self):
        
        rows = {name: [0,0] for name in self.table_names}
        with open(self.results_file) as fp:
            current_table = ""
            for i, row in enumerate(fp):
                if current_table:
                    if row=="\n":
                        rows[current_table][1] = i
                        if self.debug: print("ending line {}".format(rows[current_table][1]))
                        current_table = ""
                else:
                    for name in self.table_names:
                        if row.startswith(name):
                            current_table = name
                            rows[current_table][0] = i+2
                            if self.debug: print("found table '{}' starting line {} ".format(current_table, rows[current_table][0]), end="")
        
        tables = {}
        for name in self.table_names:
            table_skiprows = rows[name][0]
            table_nrows = rows[name][1] - table_skiprows - 1
            table_df = pd.read_csv(self.results_file, skiprows=table_skiprows, nrows=table_nrows, index_col=False, dtype=object)
            if self.debug: display(table_df)
            tables[name] = table_df
        
        return tables
    
    
    # Clean race result dataframes, cast numerical columns to integers, join points information and starting positions to driver rows
    def __clean_results_tables(self):
        
        # Merge starting position info to grid column of a race table from either quali session or previous race, optionally adding quali points column
        def merge_qualy_info(race_table, qualy_table, add_qualy_points=False):
            if add_qualy_points:
                qualy_table = qualy_table[["Driver", "Pos", "Points"]]
                qualy_table = qualy_table.rename(columns={"Pos": "Grid", "Points": "Qualify Points"})
            else:
                qualy_table = qualy_table[["Driver", "Pos"]]
                qualy_table = qualy_table.rename(columns={"Pos": "Grid"})

            race_table = race_table.merge(qualy_table, how='left', on="Driver")

            race_table["Grid"] = race_table["Grid"].fillna(-1)
            # Pandas issue where NaN values cause ints to become floats?
            race_table["Grid"] = race_table["Grid"].astype(int)
            race_table["Pos"] = race_table["Pos"].astype(int)
            race_table["Laps"] = race_table["Laps"].astype(int)

            if add_qualy_points:
                race_table["Qualify Points"] = race_table["Qualify Points"].fillna(0)
                race_table["Qualify Points"] = race_table["Qualify Points"].astype(int)

            return race_table
        
        # Strip quotes and whitespace from strings, cast position and laps to integers, drop unnamed columns and rows for non-participants
        for name, table_df in self.tables.items():

            table_df = table_df.apply(lambda s: s.str.strip(' \'"'), axis=1)
            table_df = table_df.rename(columns=lambda c: c.strip(' \'"'))
            table_df = table_df.apply(lambda s: s.str.replace('(.*\d\d*\.\d{3})0$', r'\1'), axis=1)

            table_df["Pos"] = table_df["Pos"].astype(int)
            table_df["Laps"] = table_df["Laps"].astype(int)

            table_df = table_df[table_df["Laps"] > 0]

            table_df = table_df[table_df.columns.drop(list(table_df.filter(regex='Unnamed*')))]
            table_df = table_df.dropna(how='all', axis='columns')

            self.tables[name] = table_df
            
            
        # Merge points column to both quali and race sessions, for race sessions: get attatched qualifying or previous race to get starting positions
        for name, table_df in self.tables.items():

            if name.startswith("Qualify"):

                points_column = pd.Series(np.zeros(len(table_df))).add(self.points_table["qualy_points"].astype(int), fill_value=0)
                table_df["Points"] = points_column.astype(int)

            if name.startswith("Race"):

                table_df["Time/Retired"] = table_df["Time/Retired"].str.replace('^\s*$', "")
                table_df["Consistency"] = table_df["Consistency"].str.replace('^-$', "")

                points_column = pd.Series(np.zeros(len(table_df))).add(self.points_table["points"].astype(int), fill_value=0)
                table_df["Points"] = points_column.astype(int)

                session_num = int(re.findall('(?:Qualify|Race)\s(\d+)\sresult|$', name)[0])
                qualy_table_name = "Qualify result" if session_num==1 and "Qualify result" in self.tables.keys() else "Qualify {} result".format(session_num)
                previous_race_table_name = "Race {} result".format(session_num - 1)
                if qualy_table_name in self.tables:
                    table_df = merge_qualy_info(table_df, self.tables[qualy_table_name], add_qualy_points=True)
                elif previous_race_table_name in self.tables:
                    table_df = merge_qualy_info(table_df, self.tables[previous_race_table_name])
                else:
                    table_df["Grid"] = ""    

            if self.debug: display(table_df)
            self.tables[name] = table_df
        

    # Generate quali table markdown from quali session dataframe, result is a list of lines
    def __generate_qualy_table_strings(self, table_name):

        table_df = self.tables[table_name]

        lines_buffer = [table_name, self.qualy_row_0, self.qualy_row_1]

        for i, df_row in table_df.iterrows():

            driver = df_row["Driver"]

            driver_info = self.drivers_table.loc[self.drivers_table["ign"]==driver]
            assert len(driver_info)==1, "Error looking up {} in drivers_table".format(driver)

            position = df_row["Pos"]
            number = driver_info["number"].item()
            flag = driver_info["flag"].item()
            name = driver_info["name"].item()
            team = driver_info["team"].item()
            time = df_row["Best lap"]
            if position==1:
                time = "**{}**".format(time)

            line = self.qualy_row_2.format(position, number, flag, name, team, time) if i==0 else self.qualy_row.format(position, number, flag, name, team, time)
            lines_buffer.append(line)

        lines_buffer.append(self.qualy_row_last.format(self.simresults_url))

        if self.debug: print("made {} rows for table: {}".format(len(lines_buffer), table_name))
        return lines_buffer
    
    
    def __generate_race_table_strings(self, table_name):
    
        table_df = self.tables[table_name]

        lines_buffer = [table_name, self.race_row_0, self.race_row_1]

        for i, df_row in table_df.iterrows():

            driver = df_row["Driver"]

            driver_info = self.drivers_table.loc[self.drivers_table["ign"]==driver]
            assert len(driver_info)==1, "Error looking up {} in drivers_table".format(driver)

            position = df_row["Pos"]
            number = driver_info["number"].item()
            flag = driver_info["flag"].item()
            name = driver_info["name"].item()
            team = driver_info["team"].item()
            laps = df_row["Laps"]
            points = df_row["Points"]
            timeorretired = df_row["Time/Retired"] if df_row["Time/Retired"] else "DNF"
            grid = df_row["Grid"] if df_row["Grid"]>0 else "DNQ"

            if "Qualify Points" in table_df.columns:
                points = str(df_row["Points"] + df_row["Qualify Points"]) + ("^^{}^^".format(grid) if df_row["Qualify Points"]>0 else "")

            line = self.race_row_2.format(position, number, flag, name, team, laps, timeorretired, grid, points) if i==0 else self.race_row.format(position, number, flag, name, team, laps, timeorretired, grid, points)
            lines_buffer.append(line)

        lines_buffer.append(self.race_row_last.format(self.simresults_url))

        # Find fastest lap in race results
        table_df_withtimes = table_df.copy()
        table_df_withtimes["Best lap"] = pd.to_datetime(table_df_withtimes["Best lap"], format="%M:%S.%f", errors='coerce')
        
        fastest_driver = table_df.loc[table_df_withtimes["Best lap"]==table_df_withtimes["Best lap"].min()].iloc[0]["Driver"]
        fastest_driver_flag = self.drivers_table.loc[self.drivers_table["ign"]==fastest_driver].iloc[0]["flag"]
        
        fastest_time = table_df.loc[table_df["Driver"]==fastest_driver]["Best lap"].item()
        lines_buffer.append(self.race_row_fastestlap.format(fastest_driver_flag, fastest_driver, fastest_time))

        if self.debug: print("made {} rows for table: {}".format(len(lines_buffer), table_name))
        return lines_buffer
    
    
    # Generate table markdown for all table_names, result is map from table names to table markdown string
    def generate_tables_strings(self):
        tables_strings = {}

        for name in self.table_names:
            lines_buffer = ["error!"]
            if name.startswith("Qualify"):
                lines_buffer = self.__generate_qualy_table_strings(name)
            elif name.startswith("Race"):
                lines_buffer = self.__generate_race_table_strings(name)
            tables_strings[name] = "\n".join(lines_buffer) + "\n\n"
            
        return tables_strings
    
    
    # Write table markdown for all table_names, optionally provide the generated table strings
    def write_generated_tables(self, tables_strings=None):
        if not tables_strings:
            tables_strings = self.generate_tables_strings()
            
        with open(self.output_file, "w+") as fp:
            for name in self.table_names:
                fp.write(tables_strings[name])
                if self.debug: print("wrote table {} to {}".format(name, self.output_file))

    

### Debug

In [12]:
test_series_directory = "PWCSS 2020"
test_race_directory = "nurb"
test_table_names = ["Qualify result", "Race 1 result", "Race 2 result"]

testReport = RaceReport(test_table_names, test_series_directory, test_race_directory, debug=True)
testReport.write_generated_tables()

found table 'Qualify result' starting line 15 ending line 29
found table 'Race 1 result' starting line 280 ending line 294
found table 'Race 2 result' starting line 656 ending line 670


,Pos,Class,Team,Vehicle,Driver,Laps,Best lap,Gap
0,1,"""""","""""","""ks_porsche_cayman_gt4_clubsport""","""PhotonBurst""","""6""","""01:32.9400""","""-"""
1,2,"""""","""""","""ks_porsche_cayman_gt4_clubsport""","""McLarenTim""","""6""","""01:33.2540""","""00:00.3140"""
2,3,"""""","""""","""ks_porsche_cayman_gt4_clubsport""","""Waffles""","""6""","""01:34.0250""","""00:01.0850"""
3,4,"""""","""""","""ks_porsche_cayman_gt4_clubsport""","""NotJames""","""6""","""01:34.0270""","""00:01.0870"""
4,5,"""""","""""","""ks_porsche_cayman_gt4_clubsport""","""gunciz""","""6""","""01:34.0340""","""00:01.0940"""
5,6,"""""","""""","""ks_porsche_cayman_gt4_clubsport""","""MrGrinder""","""5""","""01:34.0930""","""00:01.1530"""
6,7,"""""","""""","""ks_porsche_cayman_gt4_clubsport""","""Trempale""","""6""","""01:34.2710""","""00:01.3310"""
7,8,"""""","""""","""ks_porsche_cayman_gt4_clubsport""","""Electro""","""6""","""01:34.3650""","""00:01.4250"""
8,9,"""""","""""","""ks_porsche_cayman_gt4_clubsport""","""Georgin""","""6""","""01:34.9560""","""00:02.0160"""
9,10,"""""","""""","""ks_porsche_cayman_gt4_clubsport""","""Salzeder""","""5""","""01:35.0490""","""00:02.1090"""


,Pos,Class,Team,Vehicle,Driver,Laps,Time/Retired,Best lap,Consistency,Led,Pits,Unnamed: 11
0,1,"""""","""""","""ks_porsche_cayman_gt4_clubsport""","""PhotonBurst""","""14""",""" 21:55.1440""","""01:33.3550""","""99.83%""","""13""","""0""",
1,2,"""""","""""","""ks_porsche_cayman_gt4_clubsport""","""McLarenTim""","""14""",""" '+00:03.0720""","""01:33.1650""","""99.53%""","""0""","""0""",
2,3,"""""","""""","""ks_porsche_cayman_gt4_clubsport""","""NotJames""","""14""",""" '+00:20.0900""","""01:34.4660""","""99.59%""","""0""","""0""",
3,4,"""""","""""","""ks_porsche_cayman_gt4_clubsport""","""Trempale""","""14""",""" '+00:20.7880""","""01:34.0870""","""99.14%""","""0""","""0""",
4,5,"""""","""""","""ks_porsche_cayman_gt4_clubsport""","""gunciz""","""14""",""" '+00:27.0870""","""01:34.3720""","""98.88%""","""0""","""0""",
5,6,"""""","""""","""ks_porsche_cayman_gt4_clubsport""","""Electro""","""14""",""" '+00:27.6860""","""01:34.2960""","""99.27%""","""0""","""0""",
6,7,"""""","""""","""ks_porsche_cayman_gt4_clubsport""","""Waffles""","""14""",""" '+00:29.5310""","""01:34.3360""","""98.59%""","""0""","""0""",
7,8,"""""","""""","""ks_porsche_cayman_gt4_clubsport""","""MrGrinder""","""14""",""" '+00:29.7640""","""01:34.3580""","""99.19%""","""0""","""0""",
8,9,"""""","""""","""ks_porsche_cayman_gt4_clubsport""","""gman197002""","""14""",""" '+00:40.9500""","""01:35.6380""","""99.39%""","""0""","""0""",
9,10,"""""","""""","""ks_porsche_cayman_gt4_clubsport""","""Echo51""","""14""",""" '+00:48.0790""","""01:35.3930""","""98.83%""","""0""","""0""",


,Pos,Class,Team,Vehicle,Driver,Laps,Time/Retired,Best lap,Consistency,Led,Pits,Unnamed: 11
0,1,"""""","""""","""ks_porsche_cayman_gt4_clubsport""","""PhotonBurst""","""14""",""" 22:02.8730""","""01:33.3220""","""99.19%""","""13""","""0""",
1,2,"""""","""""","""ks_porsche_cayman_gt4_clubsport""","""McLarenTim""","""14""",""" '+00:11.2390""","""01:33.3770""","""98.83%""","""0""","""0""",
2,3,"""""","""""","""ks_porsche_cayman_gt4_clubsport""","""NotJames""","""14""",""" '+00:14.0550""","""01:34.4760""","""99.49%""","""0""","""0""",
3,4,"""""","""""","""ks_porsche_cayman_gt4_clubsport""","""Electro""","""14""",""" '+00:14.8220""","""01:34.2650""","""99.28%""","""0""","""0""",
4,5,"""""","""""","""ks_porsche_cayman_gt4_clubsport""","""gunciz""","""14""",""" '+00:16.5950""","""01:34.4330""","""99.18%""","""0""","""0""",
5,6,"""""","""""","""ks_porsche_cayman_gt4_clubsport""","""Trempale""","""14""",""" '+00:17.0010""","""01:33.8880""","""98.68%""","""0""","""0""",
6,7,"""""","""""","""ks_porsche_cayman_gt4_clubsport""","""MrGrinder""","""14""",""" '+00:20.3020""","""01:34.2810""","""99.08%""","""0""","""0""",
7,8,"""""","""""","""ks_porsche_cayman_gt4_clubsport""","""Waffles""","""14""",""" '+00:24.9390""","""01:34.3830""","""98.61%""","""0""","""0""",
8,9,"""""","""""","""ks_porsche_cayman_gt4_clubsport""","""Georgin""","""14""",""" '+00:27.1970""","""01:35.2430""","""99.45%""","""0""","""0""",
9,10,"""""","""""","""ks_porsche_cayman_gt4_clubsport""","""Salzeder""","""14""",""" '+00:39.5740""","""01:35.1270""","""98.67%""","""0""","""0""",


,Pos,Class,Team,Vehicle,Driver,Laps,Best lap,Gap,Points
0,1,,,ks_porsche_cayman_gt4_clubsport,PhotonBurst,6,01:32.940,-,3
1,2,,,ks_porsche_cayman_gt4_clubsport,McLarenTim,6,01:33.254,00:00.314,2
2,3,,,ks_porsche_cayman_gt4_clubsport,Waffles,6,01:34.025,00:01.085,1
3,4,,,ks_porsche_cayman_gt4_clubsport,NotJames,6,01:34.027,00:01.087,0
4,5,,,ks_porsche_cayman_gt4_clubsport,gunciz,6,01:34.034,00:01.094,0
5,6,,,ks_porsche_cayman_gt4_clubsport,MrGrinder,5,01:34.093,00:01.153,0
6,7,,,ks_porsche_cayman_gt4_clubsport,Trempale,6,01:34.271,00:01.331,0
7,8,,,ks_porsche_cayman_gt4_clubsport,Electro,6,01:34.365,00:01.425,0
8,9,,,ks_porsche_cayman_gt4_clubsport,Georgin,6,01:34.956,00:02.016,0
9,10,,,ks_porsche_cayman_gt4_clubsport,Salzeder,5,01:35.049,00:02.109,0


,Pos,Class,Team,Vehicle,Driver,Laps,Time/Retired,Best lap,Consistency,Led,Pits,Points,Grid,Qualify Points
0,1,,,ks_porsche_cayman_gt4_clubsport,PhotonBurst,14,21:55.144,01:33.355,99.83%,13,0,40,1,3
1,2,,,ks_porsche_cayman_gt4_clubsport,McLarenTim,14,+00:03.072,01:33.165,99.53%,0,0,36,2,2
2,3,,,ks_porsche_cayman_gt4_clubsport,NotJames,14,+00:20.090,01:34.466,99.59%,0,0,32,4,0
3,4,,,ks_porsche_cayman_gt4_clubsport,Trempale,14,+00:20.788,01:34.087,99.14%,0,0,28,7,0
4,5,,,ks_porsche_cayman_gt4_clubsport,gunciz,14,+00:27.087,01:34.372,98.88%,0,0,25,5,0
5,6,,,ks_porsche_cayman_gt4_clubsport,Electro,14,+00:27.686,01:34.296,99.27%,0,0,22,8,0
6,7,,,ks_porsche_cayman_gt4_clubsport,Waffles,14,+00:29.531,01:34.336,98.59%,0,0,19,3,1
7,8,,,ks_porsche_cayman_gt4_clubsport,MrGrinder,14,+00:29.764,01:34.358,99.19%,0,0,16,6,0
8,9,,,ks_porsche_cayman_gt4_clubsport,gman197002,14,+00:40.950,01:35.638,99.39%,0,0,13,11,0
9,10,,,ks_porsche_cayman_gt4_clubsport,Echo51,14,+00:48.079,01:35.393,98.83%,0,0,10,12,0


,Pos,Class,Team,Vehicle,Driver,Laps,Time/Retired,Best lap,Consistency,Led,Pits,Points,Grid
0,1,,,ks_porsche_cayman_gt4_clubsport,PhotonBurst,14,22:02.873,01:33.322,99.19%,13,0,40,1
1,2,,,ks_porsche_cayman_gt4_clubsport,McLarenTim,14,+00:11.239,01:33.377,98.83%,0,0,36,2
2,3,,,ks_porsche_cayman_gt4_clubsport,NotJames,14,+00:14.055,01:34.476,99.49%,0,0,32,3
3,4,,,ks_porsche_cayman_gt4_clubsport,Electro,14,+00:14.822,01:34.265,99.28%,0,0,28,6
4,5,,,ks_porsche_cayman_gt4_clubsport,gunciz,14,+00:16.595,01:34.433,99.18%,0,0,25,5
5,6,,,ks_porsche_cayman_gt4_clubsport,Trempale,14,+00:17.001,01:33.888,98.68%,0,0,22,4
6,7,,,ks_porsche_cayman_gt4_clubsport,MrGrinder,14,+00:20.302,01:34.281,99.08%,0,0,19,8
7,8,,,ks_porsche_cayman_gt4_clubsport,Waffles,14,+00:24.939,01:34.383,98.61%,0,0,16,7
8,9,,,ks_porsche_cayman_gt4_clubsport,Georgin,14,+00:27.197,01:35.243,99.45%,0,0,13,11
9,10,,,ks_porsche_cayman_gt4_clubsport,Salzeder,14,+00:39.574,01:35.127,98.67%,0,0,10,12


made 17 rows for table: Qualify result
made 18 rows for table: Race 1 result
made 18 rows for table: Race 2 result
wrote table Qualify result to PWCSS 2020/nurb/wiki_tables.txt
wrote table Race 1 result to PWCSS 2020/nurb/wiki_tables.txt
wrote table Race 2 result to PWCSS 2020/nurb/wiki_tables.txt
